In [1]:
import os
import pandas as pd
import numpy as np
from cmapPy.pandasGEXpress.parse import parse

import pickle
import pylab as plt
import statistics
import scipy.stats

In [2]:
data_year = "2020"
cell_name = "MICROGLIA-PSEN1"

data_dir = os.path.join("~/Phd_project/project_GBM/gbm_DATA/", "gbm_DATA_LINCS", data_year)

#out_dir = "00_outputs_" + data_year + "_" + cell_name
#os.makedirs(out_dir, exist_ok = True)

### open info file - compound_info

In [3]:
data_file = os.path.join(data_dir, "info", "compoundinfo_beta.txt")
compound_info_df = pd.read_csv(data_file, sep = "\t", index_col = 0)

compound_info_df = compound_info_df.sort_values(by = ["cmap_name"]).fillna("")
display(compound_info_df)

,cmap_name,target,moa,canonical_smiles,inchi_key,compound_aliases
pert_id,,,,,,
BRD-K70586315,1-EBIO,,,CCn1c2ccccc2[nH]c1=O,CXUCKELNYMZTRT-UHFFFAOYSA-N,
BRD-K62620932,1-HYDROXYANTHRAQUINONE,,,,,
BRD-K66151076,1-HYDROXYPHENAZINE,,,,,
BRD-K05741221,1-NAPHTHYLAMINE,,,,,
BRD-K72839221,1-NITRONAPHTHALENE,,,,,
...,...,...,...,...,...,...
BRD-A28095882,zonisamide,SCN11A,T-type calcium channel blocker,CN1C2CCC1CC(C2)OC(c3ccccc3)c4ccccc4N,KZFDKINRISJFCO-UHFFFAOYSA-N,
BRD-A28095882,zonisamide,SCN2A,T-type calcium channel blocker,CN1C2CCC1CC(C2)OC(c3ccccc3)c4ccccc4N,KZFDKINRISJFCO-UHFFFAOYSA-N,
BRD-A34309505,zopiclone,GABRA1,GABA receptor agonist,CN1CCN(CC1)C(=O)OC2N(C(=O)c3nccnc23)c4ccc(Cl)cn4,GBBSUAFBMRNDJC-UHFFFAOYSA-N,


In [4]:
drug_list = sorted(list(set(compound_info_df.cmap_name)))
print("Drug values in data:", len(drug_list), drug_list)

Drug values in data: 33627 ['1-EBIO', '1-HYDROXYANTHRAQUINONE', '1-HYDROXYPHENAZINE', '1-NAPHTHYLAMINE', '1-NITRONAPHTHALENE', '1-NITROPYRENE', '1-NITROSOAZOCANE', '1-benzylimidazole', '1-methylisoquinoline', '1-monopalmitin', '1-phenylbiguanide', '10-DEBC', '17-AAG', '1B', '2-AG', '2-AMINOANTHRACENE', '2-AMINOANTHRAQUINONE', '2-AMINOFLUORENE', '2-BFI', '2-CHLOROACETOPHENONE', '2-METHYLANILINE', '2-NAPHTHYLAMINE', '2-NITROPROPANE', '2-PMDQ', '2-aminobenzenesulfonamide', '2-aminopurine', '2-aminotetralin', '2-cyanopyrimidine', '2-iminobiotin', '2-iodomelatonin', '2-phenylmelatonin', "3'-fluorobenzylspiperone", '3-CPMT', '3-Cl-AHPC', '3-MPPI', '3-deazaadenosine', '3-matida', '3-methyl-GABA', '4-ACETYLAMINOBIPHENYL', '4-AMINODIPHENYL', '4-AMINOPHENOL', '4-AMINOSTILBENE', '4-HQN', '4-IBP', '4-NITROPYRENE', '4-NITROSOMORPHOLINE', '4-P-PDOT', '4-PPBP', '4-chlorophenylguanidine', '4-mu-8C', "5'-guanidinonaltrindole", '5-BDBD', '5-FP', '5-METHYLCHRYSENE', '5-NITROACENAPHTHENE', '5-iodotubercid

## Landmark genes

### open info file - gene_info

In [6]:
data_file = os.path.join(data_dir, "info", "geneinfo_beta.txt")
gene_info_df = pd.read_csv(data_file, sep = "\t", index_col = 1)

gene_info_df = gene_info_df.sort_values(by = ["gene_symbol"])
display(gene_info_df)

,gene_id,ensembl_id,gene_title,gene_type,src,feature_space
gene_symbol,,,,,,
A1CF,29974,ENSG00000148584,APOBEC1 complementation factor,protein-coding,NCBI,inferred
A2M,2,ENSG00000175899,alpha-2-macroglobulin,protein-coding,NCBI,best inferred
A4GALT,53947,ENSG00000128274,"alpha 1,4-galactosyltransferase (P blood group)",protein-coding,NCBI,best inferred
A4GNT,51146,ENSG00000118017,"alpha-1,4-N-acetylglucosaminyltransferase",protein-coding,NCBI,inferred
AAAS,8086,ENSG00000094914,aladin WD repeat nucleoporin,protein-coding,NCBI,best inferred
...,...,...,...,...,...,...
ZXDB,158586,ENSG00000198455,zinc finger X-linked duplicated B,protein-coding,NCBI,inferred
ZXDC,79364,ENSG00000070476,ZXD family zinc finger C,protein-coding,NCBI,best inferred
ZYX,7791,ENSG00000159840,zyxin,protein-coding,NCBI,best inferred


### filtered landmark genes

In [7]:
landmark_gene_symbols = gene_info_df[gene_info_df.feature_space == "landmark"].index.tolist()
print("landmark genes symbols list: ", len(landmark_gene_symbols), landmark_gene_symbols)

landmark_gene_ids = gene_info_df.loc[landmark_gene_symbols].gene_id.tolist()
print("landmark genes ids list: ", len(landmark_gene_ids), landmark_gene_ids)

display(gene_info_df.loc[landmark_gene_symbols])

landmark genes symbols list:  978 ['AARS', 'ABCB6', 'ABCC5', 'ABCF1', 'ABCF3', 'ABHD4', 'ABHD6', 'ABL1', 'ACAA1', 'ACAT2', 'ACBD3', 'ACD', 'ACLY', 'ACOT9', 'ADAM10', 'ADAT1', 'ADGRE5', 'ADGRG1', 'ADH5', 'ADI1', 'ADO', 'ADRB2', 'AGL', 'AKAP8', 'AKAP8L', 'AKR7A2', 'AKT1', 'ALAS1', 'ALDH7A1', 'ALDOA', 'ALDOC', 'AMDHD2', 'ANKRD10', 'ANO10', 'ANXA7', 'APBB2', 'APOE', 'APP', 'APPBP2', 'ARFIP2', 'ARHGAP1', 'ARHGEF12', 'ARHGEF2', 'ARID4B', 'ARID5B', 'ARL4C', 'ARNT2', 'ARPP19', 'ASAH1', 'ASCC3', 'ATF1', 'ATF5', 'ATF6', 'ATG3', 'ATMIN', 'ATP11B', 'ATP1B1', 'ATP2C1', 'ATP6V0B', 'ATP6V1D', 'AURKA', 'AURKB', 'AXIN1', 'B4GAT1', 'BACE2', 'BAD', 'BAG3', 'BAMBI', 'BAX', 'BCL2', 'BCL7B', 'BDH1', 'BECN1', 'BHLHE40', 'BID', 'BIRC2', 'BIRC5', 'BLCAP', 'BLMH', 'BLVRA', 'BMP4', 'BNIP3', 'BNIP3L', 'BPHL', 'BRCA1', 'BTK', 'BUB1B', 'BZW2', 'C2CD2', 'C2CD2L', 'C2CD5', 'C5', 'CAB39', 'CALM3', 'CALU', 'CAMSAP2', 'CANT1', 'CAPN1', 'CARMIL1', 'CASC3', 'CASK', 'CASP10', 'CASP2', 'CASP3', 'CASP7', 'CAST', 'CAT', 'CBLB

,gene_id,ensembl_id,gene_title,gene_type,src,feature_space
gene_symbol,,,,,,
AARS,16,ENSG00000090861,alanyl-tRNA synthetase,protein-coding,NCBI,landmark
ABCB6,10058,ENSG00000115657,ATP binding cassette subfamily B member 6 (Lan...,protein-coding,NCBI,landmark
ABCC5,10057,ENSG00000114770,ATP binding cassette subfamily C member 5,protein-coding,NCBI,landmark
ABCF1,23,ENSG00000204574,ATP binding cassette subfamily F member 1,protein-coding,NCBI,landmark
ABCF3,55324,ENSG00000161204,ATP binding cassette subfamily F member 3,protein-coding,NCBI,landmark
...,...,...,...,...,...,...
ZNF395,55893,ENSG00000186918,zinc finger protein 395,protein-coding,NCBI,landmark
ZNF451,26036,ENSG00000112200,zinc finger protein 451,protein-coding,NCBI,landmark
ZNF586,54807,ENSG00000083828,zinc finger protein 586,protein-coding,NCBI,landmark


## Level_3 samples

### open info file - inst_info

In [8]:
data_file = os.path.join(data_dir, "info", "instinfo_beta.txt")
inst_info_df = pd.read_csv(data_file, sep = "\t", index_col = 20)

inst_info_df = inst_info_df.sort_values(by = ["sample_id"]).fillna("")
display(inst_info_df)

/tmp/ipykernel_1196751/362428008.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  inst_info_df = pd.read_csv(data_file, sep = "\t", index_col = 20)


,bead_batch,nearest_dose,pert_dose,pert_dose_unit,pert_idose,pert_time,pert_itime,pert_time_unit,cell_mfc_name,pert_mfc_id,...,pert_id,pert_type,cell_iname,qc_pass,dyn_range,inv_level_10,build_name,failure_mode,project_code,cmap_name
sample_id,,,,,,,,,,,,,,,,,,,,,
ABY001_A375_XH_X1_B15:A03,b15,,,,,3.0,3 h,h,A375,DMSO,...,DMSO,ctl_vehicle,A375,1.0,13.4146,4400.0,,,ABY,DMSO
ABY001_A375_XH_X1_B15:A04,b15,,,,,3.0,3 h,h,A375,DMSO,...,DMSO,ctl_vehicle,A375,1.0,14.1181,3826.0,,,ABY,DMSO
ABY001_A375_XH_X1_B15:A05,b15,,,,,3.0,3 h,h,A375,DMSO,...,DMSO,ctl_vehicle,A375,1.0,16.2926,3731.0,,,ABY,DMSO
ABY001_A375_XH_X1_B15:A06,b15,,,,,3.0,3 h,h,A375,DMSO,...,DMSO,ctl_vehicle,A375,1.0,13.0177,4413.0,,,ABY,DMSO
ABY001_A375_XH_X1_B15:A07,b15,,,,,3.0,3 h,h,A375,DMSO,...,DMSO,ctl_vehicle,A375,1.0,12.5050,4408.0,,,ABY,DMSO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZTO.XPR001_U937_408H_X1_B17:N16,b17,,,,,408.0,408 h,h,U937,PDS5B,...,PDS5B,trt_xpr,U937,1.0,28.0000,3080.0,,,ZTO,PDS5B
ZTO.XPR001_U937_408H_X1_B17:N18,b17,,,,,408.0,408 h,h,U937,GC1,...,GC1,ctl_vector,U937,1.0,34.3743,3214.0,,,ZTO,GC1
ZTO.XPR001_U937_408H_X1_B17:N20,b17,,,,,408.0,408 h,h,U937,GC2,...,GC2,ctl_vector,U937,1.0,35.8750,2870.0,,,ZTO,GC2


### filtered specific cells

In [9]:
level3_cell_ids = inst_info_df[inst_info_df.cell_iname == cell_name].index.tolist()
print(cell_name, "cells in Level_3 samples list:", len(level3_cell_ids), level3_cell_ids)

MICROGLIA-PSEN1 cells in Level_3 samples list: 23 ['TSAI002_MICROGLIA-PSEN1_XH_X1_B18:A23', 'TSAI002_MICROGLIA-PSEN1_XH_X1_B18:B23', 'TSAI002_MICROGLIA-PSEN1_XH_X1_B18:B24', 'TSAI002_MICROGLIA-PSEN1_XH_X1_B18:C23', 'TSAI002_MICROGLIA-PSEN1_XH_X1_B18:D23', 'TSAI002_MICROGLIA-PSEN1_XH_X1_B18:D24', 'TSAI002_MICROGLIA-PSEN1_XH_X1_B18:E23', 'TSAI002_MICROGLIA-PSEN1_XH_X1_B18:F23', 'TSAI002_MICROGLIA-PSEN1_XH_X1_B18:F24', 'TSAI002_MICROGLIA-PSEN1_XH_X1_B18:G23', 'TSAI002_MICROGLIA-PSEN1_XH_X1_B18:H23', 'TSAI002_MICROGLIA-PSEN1_XH_X1_B18:H24', 'TSAI002_MICROGLIA-PSEN1_XH_X1_B18:I23', 'TSAI002_MICROGLIA-PSEN1_XH_X1_B18:J23', 'TSAI002_MICROGLIA-PSEN1_XH_X1_B18:K23', 'TSAI002_MICROGLIA-PSEN1_XH_X1_B18:L23', 'TSAI002_MICROGLIA-PSEN1_XH_X1_B18:L24', 'TSAI002_MICROGLIA-PSEN1_XH_X1_B18:M23', 'TSAI002_MICROGLIA-PSEN1_XH_X1_B18:N23', 'TSAI002_MICROGLIA-PSEN1_XH_X1_B18:N24', 'TSAI002_MICROGLIA-PSEN1_XH_X1_B18:O23', 'TSAI002_MICROGLIA-PSEN1_XH_X1_B18:P23', 'TSAI002_MICROGLIA-PSEN1_XH_X1_B18:P24']


In [10]:
level3_cell_ids_ctl = inst_info_df[
                                    np.logical_and(
                                                    inst_info_df.cell_iname == cell_name,
                                                    inst_info_df.pert_type.str.contains("ctl", case = False)
                                                    )
                                    ].index.tolist()

print(cell_name, "cells in Level_3 samples list, CONTROLS:", len(level3_cell_ids_ctl), level3_cell_ids_ctl)

MICROGLIA-PSEN1 cells in Level_3 samples list, CONTROLS: 3 ['TSAI002_MICROGLIA-PSEN1_XH_X1_B18:K23', 'TSAI002_MICROGLIA-PSEN1_XH_X1_B18:L23', 'TSAI002_MICROGLIA-PSEN1_XH_X1_B18:L24']


In [11]:
level3_cell_ids_trt_cp = inst_info_df[
                                        np.logical_and(
                                                        inst_info_df.cell_iname == cell_name,
                                                        inst_info_df.pert_type.str.contains("trt_cp", case = False) # or == "trt_cp"
                                                       )
                                        ].index.tolist()

print(cell_name, "cells in Level_3 samples list, TREATMENTS:" , len(level3_cell_ids_trt_cp), level3_cell_ids_trt_cp)

MICROGLIA-PSEN1 cells in Level_3 samples list, TREATMENTS: 20 ['TSAI002_MICROGLIA-PSEN1_XH_X1_B18:A23', 'TSAI002_MICROGLIA-PSEN1_XH_X1_B18:B23', 'TSAI002_MICROGLIA-PSEN1_XH_X1_B18:B24', 'TSAI002_MICROGLIA-PSEN1_XH_X1_B18:C23', 'TSAI002_MICROGLIA-PSEN1_XH_X1_B18:D23', 'TSAI002_MICROGLIA-PSEN1_XH_X1_B18:D24', 'TSAI002_MICROGLIA-PSEN1_XH_X1_B18:E23', 'TSAI002_MICROGLIA-PSEN1_XH_X1_B18:F23', 'TSAI002_MICROGLIA-PSEN1_XH_X1_B18:F24', 'TSAI002_MICROGLIA-PSEN1_XH_X1_B18:G23', 'TSAI002_MICROGLIA-PSEN1_XH_X1_B18:H23', 'TSAI002_MICROGLIA-PSEN1_XH_X1_B18:H24', 'TSAI002_MICROGLIA-PSEN1_XH_X1_B18:I23', 'TSAI002_MICROGLIA-PSEN1_XH_X1_B18:J23', 'TSAI002_MICROGLIA-PSEN1_XH_X1_B18:M23', 'TSAI002_MICROGLIA-PSEN1_XH_X1_B18:N23', 'TSAI002_MICROGLIA-PSEN1_XH_X1_B18:N24', 'TSAI002_MICROGLIA-PSEN1_XH_X1_B18:O23', 'TSAI002_MICROGLIA-PSEN1_XH_X1_B18:P23', 'TSAI002_MICROGLIA-PSEN1_XH_X1_B18:P24']


### check sum

In [12]:
len(level3_cell_ids_ctl) + len(level3_cell_ids_trt_cp) == len(level3_cell_ids)

True

### reordering

In [13]:
level3_cell_ids = level3_cell_ids_ctl + level3_cell_ids_trt_cp
print(cell_name, "cells in Level_3 samples list, CONTROLS+TREATMENTS:", len(level3_cell_ids), level3_cell_ids)

MICROGLIA-PSEN1 cells in Level_3 samples list, CONTROLS+TREATMENTS: 23 ['TSAI002_MICROGLIA-PSEN1_XH_X1_B18:K23', 'TSAI002_MICROGLIA-PSEN1_XH_X1_B18:L23', 'TSAI002_MICROGLIA-PSEN1_XH_X1_B18:L24', 'TSAI002_MICROGLIA-PSEN1_XH_X1_B18:A23', 'TSAI002_MICROGLIA-PSEN1_XH_X1_B18:B23', 'TSAI002_MICROGLIA-PSEN1_XH_X1_B18:B24', 'TSAI002_MICROGLIA-PSEN1_XH_X1_B18:C23', 'TSAI002_MICROGLIA-PSEN1_XH_X1_B18:D23', 'TSAI002_MICROGLIA-PSEN1_XH_X1_B18:D24', 'TSAI002_MICROGLIA-PSEN1_XH_X1_B18:E23', 'TSAI002_MICROGLIA-PSEN1_XH_X1_B18:F23', 'TSAI002_MICROGLIA-PSEN1_XH_X1_B18:F24', 'TSAI002_MICROGLIA-PSEN1_XH_X1_B18:G23', 'TSAI002_MICROGLIA-PSEN1_XH_X1_B18:H23', 'TSAI002_MICROGLIA-PSEN1_XH_X1_B18:H24', 'TSAI002_MICROGLIA-PSEN1_XH_X1_B18:I23', 'TSAI002_MICROGLIA-PSEN1_XH_X1_B18:J23', 'TSAI002_MICROGLIA-PSEN1_XH_X1_B18:M23', 'TSAI002_MICROGLIA-PSEN1_XH_X1_B18:N23', 'TSAI002_MICROGLIA-PSEN1_XH_X1_B18:N24', 'TSAI002_MICROGLIA-PSEN1_XH_X1_B18:O23', 'TSAI002_MICROGLIA-PSEN1_XH_X1_B18:P23', 'TSAI002_MICROGLIA-PSEN1_X

### save inst_info for specific cells

In [14]:
idx_3 = pd.Index(level3_cell_ids, name = "level_3_sample_id")
print(idx_3)

Index(['TSAI002_MICROGLIA-PSEN1_XH_X1_B18:K23',
       'TSAI002_MICROGLIA-PSEN1_XH_X1_B18:L23',
       'TSAI002_MICROGLIA-PSEN1_XH_X1_B18:L24',
       'TSAI002_MICROGLIA-PSEN1_XH_X1_B18:A23',
       'TSAI002_MICROGLIA-PSEN1_XH_X1_B18:B23',
       'TSAI002_MICROGLIA-PSEN1_XH_X1_B18:B24',
       'TSAI002_MICROGLIA-PSEN1_XH_X1_B18:C23',
       'TSAI002_MICROGLIA-PSEN1_XH_X1_B18:D23',
       'TSAI002_MICROGLIA-PSEN1_XH_X1_B18:D24',
       'TSAI002_MICROGLIA-PSEN1_XH_X1_B18:E23',
       'TSAI002_MICROGLIA-PSEN1_XH_X1_B18:F23',
       'TSAI002_MICROGLIA-PSEN1_XH_X1_B18:F24',
       'TSAI002_MICROGLIA-PSEN1_XH_X1_B18:G23',
       'TSAI002_MICROGLIA-PSEN1_XH_X1_B18:H23',
       'TSAI002_MICROGLIA-PSEN1_XH_X1_B18:H24',
       'TSAI002_MICROGLIA-PSEN1_XH_X1_B18:I23',
       'TSAI002_MICROGLIA-PSEN1_XH_X1_B18:J23',
       'TSAI002_MICROGLIA-PSEN1_XH_X1_B18:M23',
       'TSAI002_MICROGLIA-PSEN1_XH_X1_B18:N23',
       'TSAI002_MICROGLIA-PSEN1_XH_X1_B18:N24',
       'TSAI002_MICROGLIA-PSEN1_XH_X1_B1

In [15]:
 
inst_info_cell_df = pd.DataFrame({"cell" : inst_info_df.loc[level3_cell_ids, "cell_iname"],
                                 "time" : inst_info_df.loc[level3_cell_ids, "pert_itime"],
                                 "pert_type" : inst_info_df.loc[level3_cell_ids, "pert_type"],
                                 "pert_drug" : inst_info_df.loc[level3_cell_ids, "cmap_name"],
                                 "dose" : inst_info_df.loc[level3_cell_ids, "pert_idose"],
                                }, index = idx_3)

# dose for DMSO = 0 uM
inst_info_cell_df.loc[inst_info_cell_df[inst_info_cell_df.pert_drug == "DMSO"].index, "dose"] = "0 uM"

display(inst_info_cell_df)

,cell,time,pert_type,pert_drug,dose
level_3_sample_id,,,,,
TSAI002_MICROGLIA-PSEN1_XH_X1_B18:K23,MICROGLIA-PSEN1,,ctl_vehicle,DMSO,0 uM
TSAI002_MICROGLIA-PSEN1_XH_X1_B18:L23,MICROGLIA-PSEN1,,ctl_vehicle,DMSO,0 uM
TSAI002_MICROGLIA-PSEN1_XH_X1_B18:L24,MICROGLIA-PSEN1,,ctl_vehicle,DMSO,0 uM
TSAI002_MICROGLIA-PSEN1_XH_X1_B18:A23,MICROGLIA-PSEN1,,trt_cp,CI-994,10 uM
TSAI002_MICROGLIA-PSEN1_XH_X1_B18:B23,MICROGLIA-PSEN1,,trt_cp,CI-994,10 uM
TSAI002_MICROGLIA-PSEN1_XH_X1_B18:B24,MICROGLIA-PSEN1,,trt_cp,CI-994,10 uM
TSAI002_MICROGLIA-PSEN1_XH_X1_B18:C23,MICROGLIA-PSEN1,,trt_cp,943,10 uM
TSAI002_MICROGLIA-PSEN1_XH_X1_B18:D23,MICROGLIA-PSEN1,,trt_cp,943,10 uM
TSAI002_MICROGLIA-PSEN1_XH_X1_B18:D24,MICROGLIA-PSEN1,,trt_cp,943,10 uM


In [16]:
cell_list = list(set(inst_info_cell_df.cell))
print("Cell values in data:", len(cell_list), cell_list)

time_list = list(set(inst_info_cell_df.time))
print("Time values in data:", len(time_list), time_list)

pert_type_list = list(set(inst_info_cell_df.pert_type))
print("Perturbation_type values in data:", len(pert_type_list), pert_type_list)

pert_drug_list = sorted(list(set(inst_info_cell_df.pert_drug)))
print("Perturbation_drug values in data:", len(pert_drug_list), pert_drug_list)

dose_list = sorted(list(set(inst_info_cell_df.dose)))
print("Dose values in data:", len(dose_list), dose_list)

Cell values in data: 1 ['MICROGLIA-PSEN1']
Time values in data: 1 ['']
Perturbation_type values in data: 2 ['ctl_vehicle', 'trt_cp']
Perturbation_drug values in data: 10 ['943', '949', 'BRD-A19037878', 'C646', 'CI-994', 'DMSO', 'SRT-3657', 'compe', 'geldanamycin', 'wortmannin']
Dose values in data: 4 ['0 uM', '10 uM', '2.22 uM', '4 uM']


In [ ]:
inst_info_cell_df.to_excel(os.path.join(out_dir, "inst_info_" + data_year + "_" + cell_name + ".xlsx"))
inst_info_cell_df.to_csv(os.path.join(out_dir, "inst_info_" + data_year + "_" + cell_name + ".csv"))

## Open gctx files using parse

### (cmap python package) 
https://github.com/cmap/cmapPy/blob/master/tutorials/cmapPy_pandasGEXpress_tutorial.ipynb

### DATA Controls:

In [ ]:
data_file = os.path.join(data_dir, "level_3", "level3_beta_ctl_n188708x12328.gctx")

level3_cell_ctl_gctoo = parse(data_file, cid = level3_cell_ids_ctl)   #  does not work with rid = landmark_gene_ids

level3_cell_ctl_df = level3_cell_ctl_gctoo.data_df.T

# rid in file dtype = object  ---> dtype = int64 
level3_cell_ctl_df.columns = pd.to_numeric(level3_cell_ctl_df.columns)

display(level3_cell_ctl_df)
print(level3_cell_ctl_df.columns)

rid,10,100,1000,10000,10001,10003,10004,10005,10006,10007,...,9985,9987,9988,9989,999,9990,9991,9992,9993,9997
cid,,,,,,,,,,,,,,,,,,,,,
LJP007_HUVEC_24H_X1_B21:A05,6.24140,9.111850,7.27810,9.117100,8.650000,6.33040,6.134400,7.638801,7.241900,9.230400,...,5.536400,10.772500,7.854850,8.363000,0.57700,5.50530,6.782700,5.290000,7.141300,8.888000
LJP007_HUVEC_24H_X1_B21:A06,6.39295,7.940701,7.23420,8.809700,8.573000,6.20470,5.913300,7.700500,6.764300,9.258850,...,5.845901,10.401300,7.866400,8.853500,1.23050,4.74500,6.141500,4.982100,7.819800,9.241400
LJP007_HUVEC_24H_X1_B21:B05,7.55420,9.289900,7.04930,9.625300,8.654949,6.99675,7.021500,7.045500,6.608900,9.006100,...,5.701400,10.004100,7.374500,9.097000,0.48540,4.97210,6.212700,5.321450,7.816599,7.944600
LJP007_HUVEC_24H_X1_B21:B06,6.04730,9.257700,7.33250,8.905500,8.578500,6.52850,6.314200,7.450750,7.127500,9.305500,...,4.447200,10.505900,7.656700,9.618800,1.63090,5.32760,6.070900,5.162700,7.897900,8.246700
LJP007_HUVEC_24H_X1_B21:F07,6.57020,9.542100,7.74780,8.600900,7.844800,6.36720,5.518600,7.347900,7.074400,8.979100,...,5.478900,10.923000,7.795600,9.370300,0.00000,5.11710,6.597650,4.945300,7.196700,8.809450
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
REP.B024_HUVEC.A_24H_X2_B25:J14,6.13930,7.568450,10.11145,9.815950,9.359600,6.06040,6.155150,7.087050,7.024500,9.155325,...,5.487050,10.172750,8.167575,8.351075,1.53865,4.84080,6.020175,5.123250,7.079500,7.209325
REP.B024_HUVEC.A_24H_X2_B25:J15,5.60800,7.382250,9.79115,9.271600,8.579451,6.00075,5.022200,7.083050,6.039250,8.966050,...,4.636400,10.028925,6.965500,8.964800,0.99435,5.33190,5.928800,5.363075,7.993251,7.054300
REP.B024_HUVEC.A_24H_X2_B25:J16,6.46905,8.008349,9.18445,10.431900,9.416550,7.25335,5.598600,6.267150,7.879000,9.167351,...,4.759400,9.330300,8.303600,8.499400,1.53865,4.73695,6.240450,4.923100,7.801050,8.034850


Index([   10,   100,  1000, 10000, 10001, 10003, 10004, 10005, 10006, 10007,
       ...
        9985,  9987,  9988,  9989,   999,  9990,  9991,  9992,  9993,  9997],
      dtype='int64', name='rid', length=12328)


### DATA Treatments:

In [ ]:
# data_file = os.path.join(data_dir, "level_3", "level3_beta_trt_cp_n1805898x12328.gctx")

# level3_cell_trt_cp_gctoo = parse(data_file, cid = level3_cell_ids_trt_cp)

# level3_cell_trt_cp_df = level3_cell_trt_cp_gctoo.data_df.T
# level3_cell_trt_cp_df.columns = pd.to_numeric(level3_cell_trt_cp_df.columns)

# display(level3_cell_trt_cp_df)
# print(level3_cell_trt_cp_df.columns)

In [ ]:
data_file = os.path.join(data_dir, "level_3", "level3_beta_trt_cp_n1805898x12328.gctx")

#  does not open the full file (82Gb), so we divide it into 2 parts [:7000] and [7000:]

#1
level3_cell_trt_cp_gctoo_1 = parse(data_file, cid = level3_cell_ids_trt_cp[:7000])

level3_cell_trt_cp_df = level3_cell_trt_cp_gctoo_1.data_df.T
level3_cell_trt_cp_df.columns = pd.to_numeric(level3_cell_trt_cp_df.columns)

display(level3_cell_trt_cp_df)
print(level3_cell_trt_cp_df.columns)

#2
level3_cell_trt_cp_gctoo_2 = parse(data_file, cid = level3_cell_ids_trt_cp[7000:])

level3_cell_trt_cp_df_2 = level3_cell_trt_cp_gctoo_2.data_df.T
level3_cell_trt_cp_df_2.columns = pd.to_numeric(level3_cell_trt_cp_df_2.columns)

display(level3_cell_trt_cp_df_2)
print(level3_cell_trt_cp_df_2.columns)

rid,10,100,1000,10000,10001,10003,10004,10005,10006,10007,...,9985,9987,9988,9989,999,9990,9991,9992,9993,9997
cid,,,,,,,,,,,,,,,,,,,,,
LJP007_HUVEC_24H_X1_B21:A03,6.31100,9.90880,4.578600,9.32910,8.80610,5.6526,6.61330,8.1311,7.245200,8.6276,...,5.46265,9.535049,7.66160,6.81860,0.0000,5.24700,4.167000,5.38700,8.437450,7.7318
LJP007_HUVEC_24H_X1_B21:A04,6.30575,8.21035,5.040000,7.82630,8.38720,6.6026,6.30710,6.8521,7.684599,10.1481,...,6.45615,10.020900,8.13755,8.32570,1.5383,5.34860,5.817000,5.22785,7.921100,9.7457
LJP007_HUVEC_24H_X1_B21:A07,6.30050,8.26180,6.392800,10.39250,7.55270,6.3170,5.61320,6.5571,8.107500,8.5451,...,4.92550,9.165600,8.98500,8.92260,5.3809,6.09380,4.685500,4.95950,8.084800,9.5164
LJP007_HUVEC_24H_X1_B21:A08,5.92480,7.81910,7.897150,8.74900,8.57395,6.2812,5.91310,7.0880,6.957500,8.6821,...,5.38670,10.020100,7.65030,9.07150,1.4445,5.19680,5.508200,5.36470,7.820499,8.6657
LJP007_HUVEC_24H_X1_B21:A09,7.34470,8.91630,7.901899,9.85660,8.49460,7.0279,5.72320,7.8077,6.489300,9.0739,...,5.39375,10.308800,8.89190,9.05880,1.8551,5.56400,5.624750,5.79965,7.526700,8.1621
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
REP.A020_HUVEC.A_24H_X3_B25:I14,6.21980,9.20230,8.007800,8.70700,8.01995,6.1717,4.90730,6.6164,6.891699,9.5851,...,4.23360,9.828100,8.60810,9.02920,2.2268,5.12000,6.817501,5.27630,7.680600,7.3856
REP.A020_HUVEC.A_24H_X3_B25:I15,6.03135,8.85930,8.490200,8.32810,8.28125,6.0566,5.07310,6.7991,6.418300,9.5341,...,4.85845,10.251700,8.39505,7.75335,2.0428,5.61320,6.278900,5.16210,6.954400,7.7419
REP.A020_HUVEC.A_24H_X3_B25:I16,6.16580,9.17230,10.547400,9.30190,7.85290,6.1501,4.76410,6.9973,6.300300,9.6186,...,6.61220,9.891100,7.92030,7.04850,2.7679,5.59220,6.273400,5.38380,7.843050,7.6148


Index([   10,   100,  1000, 10000, 10001, 10003, 10004, 10005, 10006, 10007,
       ...
        9985,  9987,  9988,  9989,   999,  9990,  9991,  9992,  9993,  9997],
      dtype='int64', name='rid', length=12328)


rid,10,100,1000,10000,10001,10003,10004,10005,10006,10007,...,9985,9987,9988,9989,999,9990,9991,9992,9993,9997
cid,,,,,,,,,,,,,,,,,,,,,
REP.A020_HUVEC.A_24H_X3_B25:I20,8.279400,8.295000,10.19770,9.582200,8.695400,6.823500,4.619600,7.59600,7.347700,9.409300,...,3.43370,9.522600,7.496700,8.560100,4.12760,4.91120,6.33470,5.071500,8.034500,6.021000
REP.A020_HUVEC.A_24H_X3_B25:I21,7.314250,9.902000,8.57110,8.408300,7.866400,6.060550,5.190100,6.84120,7.009399,8.732599,...,5.06565,9.691500,8.033800,8.604100,2.17690,4.81460,6.00740,5.279750,7.504550,7.072400
REP.A020_HUVEC.A_24H_X3_B25:I22,6.701200,8.498400,10.46420,7.795000,8.306300,6.189700,5.581100,6.25750,7.836900,9.272600,...,5.27905,10.320800,8.453501,8.653800,2.97700,5.18780,6.08770,5.342400,8.795750,6.596650
REP.A020_HUVEC.A_24H_X3_B25:I23,8.781200,10.807400,10.51090,9.437700,9.324400,6.156800,4.948800,6.82140,7.372900,9.140400,...,4.73700,9.079750,8.134700,7.608800,4.55180,5.42135,5.69950,6.054300,7.715500,7.991000
REP.A020_HUVEC.A_24H_X3_B25:I24,5.754700,8.102800,9.38300,8.881400,7.808700,6.767700,4.772800,6.89635,6.305350,9.328400,...,5.17170,9.720400,8.484200,8.268101,0.98290,4.64730,6.17330,4.655500,7.570300,7.825550
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
REP.B024_HUVEC.A_24H_X2_B25:P20,5.977600,8.190300,7.53225,10.356100,8.210800,6.396700,5.057800,6.85570,6.557650,9.620600,...,5.22185,10.182751,8.539551,8.670700,1.53865,5.51680,5.28260,5.457550,7.821650,8.373500
REP.B024_HUVEC.A_24H_X2_B25:P21,5.925050,7.291200,9.17790,11.078850,8.174825,6.444150,5.774500,6.95550,6.652200,8.768250,...,4.98975,9.800400,8.077850,9.398475,0.91135,5.03470,6.08970,5.251625,8.712325,7.931700
REP.B024_HUVEC.A_24H_X2_B25:P22,7.150275,7.365350,8.64630,10.228476,8.516400,6.273550,5.846350,7.18660,6.881450,8.691700,...,4.75040,9.704100,8.293150,8.740351,2.77525,4.18105,5.15975,5.713150,7.403100,8.531799


Index([   10,   100,  1000, 10000, 10001, 10003, 10004, 10005, 10006, 10007,
       ...
        9985,  9987,  9988,  9989,   999,  9990,  9991,  9992,  9993,  9997],
      dtype='int64', name='rid', length=12328)


### DATA, unite all:

In [ ]:
# unite parts into 1 DataFrame 
# concat makes the same order of columns like in the first dataframe

level3_cell_df = pd.concat([
                            level3_cell_ctl_df,
                            level3_cell_trt_cp_df,
                            level3_cell_trt_cp_df_2  # delete
                            ], axis = 0)
display(level3_cell_df)

rid,10,100,1000,10000,10001,10003,10004,10005,10006,10007,...,9985,9987,9988,9989,999,9990,9991,9992,9993,9997
cid,,,,,,,,,,,,,,,,,,,,,
LJP007_HUVEC_24H_X1_B21:A05,6.241400,9.111850,7.27810,9.117100,8.650000,6.330400,6.134400,7.638801,7.241900,9.230400,...,5.536400,10.772500,7.854850,8.363000,0.57700,5.50530,6.78270,5.290000,7.141300,8.888000
LJP007_HUVEC_24H_X1_B21:A06,6.392950,7.940701,7.23420,8.809700,8.573000,6.204700,5.913300,7.700500,6.764300,9.258850,...,5.845901,10.401300,7.866400,8.853500,1.23050,4.74500,6.14150,4.982100,7.819800,9.241400
LJP007_HUVEC_24H_X1_B21:B05,7.554200,9.289900,7.04930,9.625300,8.654949,6.996750,7.021500,7.045500,6.608900,9.006100,...,5.701400,10.004100,7.374500,9.097000,0.48540,4.97210,6.21270,5.321450,7.816599,7.944600
LJP007_HUVEC_24H_X1_B21:B06,6.047300,9.257700,7.33250,8.905500,8.578500,6.528500,6.314200,7.450750,7.127500,9.305500,...,4.447200,10.505900,7.656700,9.618800,1.63090,5.32760,6.07090,5.162700,7.897900,8.246700
LJP007_HUVEC_24H_X1_B21:F07,6.570200,9.542100,7.74780,8.600900,7.844800,6.367200,5.518600,7.347900,7.074400,8.979100,...,5.478900,10.923000,7.795600,9.370300,0.00000,5.11710,6.59765,4.945300,7.196700,8.809450
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
REP.B024_HUVEC.A_24H_X2_B25:P20,5.977600,8.190300,7.53225,10.356100,8.210800,6.396700,5.057800,6.855700,6.557650,9.620600,...,5.221850,10.182751,8.539551,8.670700,1.53865,5.51680,5.28260,5.457550,7.821650,8.373500
REP.B024_HUVEC.A_24H_X2_B25:P21,5.925050,7.291200,9.17790,11.078850,8.174825,6.444150,5.774500,6.955500,6.652200,8.768250,...,4.989750,9.800400,8.077850,9.398475,0.91135,5.03470,6.08970,5.251625,8.712325,7.931700
REP.B024_HUVEC.A_24H_X2_B25:P22,7.150275,7.365350,8.64630,10.228476,8.516400,6.273550,5.846350,7.186600,6.881450,8.691700,...,4.750400,9.704100,8.293150,8.740351,2.77525,4.18105,5.15975,5.713150,7.403100,8.531799


### filtred landmark genes

In [ ]:
level3_cell_lm_df = level3_cell_df.loc[:, landmark_gene_ids]

# order important
level3_cell_lm_df.columns = landmark_gene_symbols

display(level3_cell_lm_df)

,AARS,ABCB6,ABCC5,ABCF1,ABCF3,ABHD4,ABHD6,ABL1,ACAA1,ACAT2,...,ZMIZ1,ZMYM2,ZNF131,ZNF274,ZNF318,ZNF395,ZNF451,ZNF586,ZNF589,ZW10
cid,,,,,,,,,,,,,,,,,,,,,
LJP007_HUVEC_24H_X1_B21:A05,9.541850,8.185801,5.608300,10.143400,8.704700,6.391950,6.571150,10.377900,7.105300,8.638500,...,9.75470,9.49890,8.027250,10.108601,6.479600,8.53980,8.074600,6.323100,7.085300,9.017550
LJP007_HUVEC_24H_X1_B21:A06,9.485550,8.554100,5.824650,10.229400,8.386950,6.210500,6.323500,10.105300,7.229950,8.535100,...,9.76060,9.49970,7.842400,10.324150,6.684550,9.58740,8.150850,6.072200,7.124100,9.121500
LJP007_HUVEC_24H_X1_B21:B05,9.280100,7.537300,4.749800,10.398000,8.155701,5.886500,6.339000,10.014800,6.610000,8.744400,...,9.54490,9.12730,7.760600,10.007400,6.942500,10.72620,8.129800,6.074000,6.706200,8.853500
LJP007_HUVEC_24H_X1_B21:B06,9.314100,7.857900,5.083800,10.158600,8.548600,6.103100,6.135550,10.337500,6.833200,8.499450,...,10.31460,9.79740,7.958600,9.992400,6.384000,9.29160,7.871400,6.004050,7.023600,9.243000
LJP007_HUVEC_24H_X1_B21:F07,9.622700,8.270200,5.332200,9.982901,8.555800,6.168400,6.493400,10.228400,6.676700,7.942050,...,10.48405,9.66210,7.912050,9.744400,6.854950,9.22670,8.045099,6.027200,7.486700,9.014000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
REP.B024_HUVEC.A_24H_X2_B25:P20,7.936100,7.787575,4.473175,11.218200,8.547501,7.270425,6.112675,10.417875,6.017550,7.009550,...,11.05645,6.57840,8.350800,10.048349,6.051750,10.78325,6.310350,5.431025,5.431025,8.873125
REP.B024_HUVEC.A_24H_X2_B25:P21,8.941750,6.488150,4.760575,11.209650,8.474951,9.084325,10.236600,10.045250,7.932950,7.247825,...,10.31460,6.04330,7.734674,10.112850,5.910475,10.72080,6.695850,11.254049,6.043300,8.552650
REP.B024_HUVEC.A_24H_X2_B25:P22,9.231550,6.544500,4.445100,9.569350,8.285550,7.111125,6.703775,10.070800,6.924125,7.610000,...,9.79970,5.72245,7.871575,10.387350,6.065450,11.05345,6.139925,5.439625,5.935650,9.103400


## Level_5 signatures

### open info file - sig_info

In [ ]:
data_file = os.path.join(data_dir, "info", "siginfo_beta.txt")
sig_info_df = pd.read_csv(data_file, sep = "\t", index_col = 25)

sig_info_df = sig_info_df.sort_values(by = ["sig_id"]).fillna("")
display(sig_info_df)

C:\Users\annas\AppData\Local\Temp\ipykernel_19920\3118531704.py:2: DtypeWarning: Columns (0,3,4,28,29) have mixed types. Specify dtype option on import or set low_memory=False.
  sig_info_df = pd.read_csv(data_file, sep = "\t", index_col = 25)


,bead_batch,nearest_dose,pert_dose,pert_dose_unit,pert_idose,pert_itime,pert_time,pert_time_unit,cell_mfc_name,pert_mfc_id,...,cell_iname,det_wells,det_plates,distil_ids,build_name,project_code,cmap_name,is_exemplar_sig,is_ncs_sig,is_null_sig
sig_id,,,,,,,,,,,,,,,,,,,,,
ABY001_A375_XH:ADO-TRASTUZUMAB_EMTANSINE:0.3125:24,b15,,0.3125,ug/ml,0.31 ug/ml,24 h,24.0,h,A375,ADO-TRASTUZUMAB EMTANSINE,...,A375,P15|P19|P23,ABY001_A375_XH_X1_B15,ABY001_A375_XH_X1_B15:P15|ABY001_A375_XH_X1_B1...,,ABY,ADO-TRASTUZUMAB EMTANSINE,0,0.0,0.0
ABY001_A375_XH:ADO-TRASTUZUMAB_EMTANSINE:0.3125:3,b15,,0.3125,ug/ml,0.31 ug/ml,3 h,3.0,h,A375,ADO-TRASTUZUMAB EMTANSINE,...,A375,H03|H07|H11,ABY001_A375_XH_X1_B15,ABY001_A375_XH_X1_B15:H03|ABY001_A375_XH_X1_B1...,,ABY,ADO-TRASTUZUMAB EMTANSINE,0,0.0,0.0
ABY001_A375_XH:ADO-TRASTUZUMAB_EMTANSINE:0.625:24,b15,,0.625,ug/ml,0.63 ug/ml,24 h,24.0,h,A375,ADO-TRASTUZUMAB EMTANSINE,...,A375,O15|O19|O23,ABY001_A375_XH_X1_B15,ABY001_A375_XH_X1_B15:O15|ABY001_A375_XH_X1_B1...,,ABY,ADO-TRASTUZUMAB EMTANSINE,0,0.0,0.0
ABY001_A375_XH:ADO-TRASTUZUMAB_EMTANSINE:0.625:3,b15,,0.625,ug/ml,0.63 ug/ml,3 h,3.0,h,A375,ADO-TRASTUZUMAB EMTANSINE,...,A375,G03|G07|G11,ABY001_A375_XH_X1_B15,ABY001_A375_XH_X1_B15:G03|ABY001_A375_XH_X1_B1...,,ABY,ADO-TRASTUZUMAB EMTANSINE,0,0.0,0.0
ABY001_A375_XH:ADO-TRASTUZUMAB_EMTANSINE:1.25:24,b15,,1.25,ug/ml,1.25 ug/ml,24 h,24.0,h,A375,ADO-TRASTUZUMAB EMTANSINE,...,A375,N15|N19|N23,ABY001_A375_XH_X1_B15,ABY001_A375_XH_X1_B15:N15|ABY001_A375_XH_X1_B1...,,ABY,ADO-TRASTUZUMAB EMTANSINE,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZTO.XPR001_U937_408H:PDS5B:-666,b17,,,,,408 h,408.0,h,U937,PDS5B,...,U937,E15|E16|F15|F16|G15|G16|H15|H16|I15|I16|J15|J1...,ZTO.XPR001_U937_408H_X1_B17,ZTO.XPR001_U937_408H_X1_B17:E15|ZTO.XPR001_U93...,,ZTO,PDS5B,0,0.0,0.0
ZTO.XPR001_U937_408H:RAD21:-666,b17,,,,,408 h,408.0,h,U937,RAD21,...,U937,E11|E12|F11|F12|G11|G12|H11|H12|I11|I12|J11|J1...,ZTO.XPR001_U937_408H_X1_B17,ZTO.XPR001_U937_408H_X1_B17:E11|ZTO.XPR001_U93...,,ZTO,RAD21,0,0.0,0.0
ZTO.XPR001_U937_408H:SMC1A:-666,b17,,,,,408 h,408.0,h,U937,SMC1A,...,U937,E07|E08|F07|F08|G07|G08|H07|H08|I07|I08|J07|J0...,ZTO.XPR001_U937_408H_X1_B17,ZTO.XPR001_U937_408H_X1_B17:E07|ZTO.XPR001_U93...,,ZTO,SMC1A,0,0.0,0.0


### filtered specific cells

In [ ]:
level5_cell_ids = sig_info_df[sig_info_df.cell_iname == cell_name].index.tolist()
print(cell_name, "cells in Level_5 signatures list:", len(level5_cell_ids), level5_cell_ids)

HUVEC cells in Level_5 signatures list: 8154 ['LJP007_HUVEC_24H:A03', 'LJP007_HUVEC_24H:A04', 'LJP007_HUVEC_24H:A05', 'LJP007_HUVEC_24H:A06', 'LJP007_HUVEC_24H:A07', 'LJP007_HUVEC_24H:A08', 'LJP007_HUVEC_24H:A09', 'LJP007_HUVEC_24H:A10', 'LJP007_HUVEC_24H:A11', 'LJP007_HUVEC_24H:A12', 'LJP007_HUVEC_24H:A13', 'LJP007_HUVEC_24H:A14', 'LJP007_HUVEC_24H:A15', 'LJP007_HUVEC_24H:A16', 'LJP007_HUVEC_24H:A17', 'LJP007_HUVEC_24H:A18', 'LJP007_HUVEC_24H:A19', 'LJP007_HUVEC_24H:A20', 'LJP007_HUVEC_24H:A21', 'LJP007_HUVEC_24H:A22', 'LJP007_HUVEC_24H:A23', 'LJP007_HUVEC_24H:A24', 'LJP007_HUVEC_24H:B03', 'LJP007_HUVEC_24H:B04', 'LJP007_HUVEC_24H:B05', 'LJP007_HUVEC_24H:B06', 'LJP007_HUVEC_24H:B07', 'LJP007_HUVEC_24H:B08', 'LJP007_HUVEC_24H:B09', 'LJP007_HUVEC_24H:B10', 'LJP007_HUVEC_24H:B11', 'LJP007_HUVEC_24H:B12', 'LJP007_HUVEC_24H:B13', 'LJP007_HUVEC_24H:B14', 'LJP007_HUVEC_24H:B15', 'LJP007_HUVEC_24H:B16', 'LJP007_HUVEC_24H:B17', 'LJP007_HUVEC_24H:B18', 'LJP007_HUVEC_24H:B19', 'LJP007_HUVEC_24H:

In [ ]:
level5_cell_ids_ctl = sig_info_df[
                                    np.logical_and(
                                                    sig_info_df.cell_iname == cell_name,
                                                    sig_info_df.pert_type.str.contains("ctl", case = False)
                                                    )
                                    ].index.tolist()

print(cell_name, "cells in Level_5 signatures list, CONTROLS:", len(level5_cell_ids_ctl), level5_cell_ids_ctl)

HUVEC cells in Level_5 signatures list, CONTROLS: 458 ['LJP007_HUVEC_24H:A05', 'LJP007_HUVEC_24H:A06', 'LJP007_HUVEC_24H:B05', 'LJP007_HUVEC_24H:B06', 'LJP007_HUVEC_24H:F07', 'LJP007_HUVEC_24H:F08', 'LJP007_HUVEC_24H:F09', 'LJP007_HUVEC_24H:F10', 'LJP007_HUVEC_24H:F11', 'LJP007_HUVEC_24H:F12', 'LJP007_HUVEC_24H:J13', 'LJP007_HUVEC_24H:J14', 'LJP007_HUVEC_24H:J15', 'LJP007_HUVEC_24H:J16', 'LJP007_HUVEC_24H:J17', 'LJP007_HUVEC_24H:J18', 'LJP007_HUVEC_24H:N19', 'LJP007_HUVEC_24H:N20', 'LJP007_HUVEC_24H:N21', 'LJP007_HUVEC_24H:N22', 'LJP007_HUVEC_24H:N23', 'LJP007_HUVEC_24H:N24', 'LJP008_HUVEC.A_24H:A05', 'LJP008_HUVEC.A_24H:A06', 'LJP008_HUVEC.A_24H:B05', 'LJP008_HUVEC.A_24H:B06', 'LJP008_HUVEC.A_24H:F07', 'LJP008_HUVEC.A_24H:F08', 'LJP008_HUVEC.A_24H:F09', 'LJP008_HUVEC.A_24H:F10', 'LJP008_HUVEC.A_24H:F11', 'LJP008_HUVEC.A_24H:F12', 'LJP008_HUVEC.A_24H:J13', 'LJP008_HUVEC.A_24H:J14', 'LJP008_HUVEC.A_24H:J15', 'LJP008_HUVEC.A_24H:J16', 'LJP008_HUVEC.A_24H:J17', 'LJP008_HUVEC.A_24H:J18', '

In [ ]:
level5_cell_ids_trt_cp = sig_info_df[
                                        np.logical_and(
                                                        sig_info_df.cell_iname == cell_name,
                                                        sig_info_df.pert_type.str.contains("trt_cp", case = False) # or == "trt_cp"
                                                        )
                                        ].index.tolist()

print(cell_name, "cells in Level_5 signatures list, TREATMENTS:", len(level5_cell_ids_trt_cp), level5_cell_ids_trt_cp)

HUVEC cells in Level_5 signatures list, TREATMENTS: 7696 ['LJP007_HUVEC_24H:A03', 'LJP007_HUVEC_24H:A04', 'LJP007_HUVEC_24H:A07', 'LJP007_HUVEC_24H:A08', 'LJP007_HUVEC_24H:A09', 'LJP007_HUVEC_24H:A10', 'LJP007_HUVEC_24H:A11', 'LJP007_HUVEC_24H:A12', 'LJP007_HUVEC_24H:A13', 'LJP007_HUVEC_24H:A14', 'LJP007_HUVEC_24H:A15', 'LJP007_HUVEC_24H:A16', 'LJP007_HUVEC_24H:A17', 'LJP007_HUVEC_24H:A18', 'LJP007_HUVEC_24H:A19', 'LJP007_HUVEC_24H:A20', 'LJP007_HUVEC_24H:A21', 'LJP007_HUVEC_24H:A22', 'LJP007_HUVEC_24H:A23', 'LJP007_HUVEC_24H:A24', 'LJP007_HUVEC_24H:B03', 'LJP007_HUVEC_24H:B04', 'LJP007_HUVEC_24H:B07', 'LJP007_HUVEC_24H:B08', 'LJP007_HUVEC_24H:B09', 'LJP007_HUVEC_24H:B10', 'LJP007_HUVEC_24H:B11', 'LJP007_HUVEC_24H:B12', 'LJP007_HUVEC_24H:B13', 'LJP007_HUVEC_24H:B14', 'LJP007_HUVEC_24H:B15', 'LJP007_HUVEC_24H:B16', 'LJP007_HUVEC_24H:B17', 'LJP007_HUVEC_24H:B18', 'LJP007_HUVEC_24H:B19', 'LJP007_HUVEC_24H:B20', 'LJP007_HUVEC_24H:B21', 'LJP007_HUVEC_24H:B22', 'LJP007_HUVEC_24H:B23', 'LJP00

### check sum

In [ ]:
len(level5_cell_ids_ctl) + len(level5_cell_ids_trt_cp) == len(level5_cell_ids)

True

### reordering

In [ ]:
level5_cell_ids = level5_cell_ids_ctl + level5_cell_ids_trt_cp
print(cell_name, "cells in Level_5 signatures list, CONTROLS+TREATMENTS:", len(level5_cell_ids), level5_cell_ids)

HUVEC cells in Level_5 signatures list, CONTROLS+TREATMENTS: 8154 ['LJP007_HUVEC_24H:A05', 'LJP007_HUVEC_24H:A06', 'LJP007_HUVEC_24H:B05', 'LJP007_HUVEC_24H:B06', 'LJP007_HUVEC_24H:F07', 'LJP007_HUVEC_24H:F08', 'LJP007_HUVEC_24H:F09', 'LJP007_HUVEC_24H:F10', 'LJP007_HUVEC_24H:F11', 'LJP007_HUVEC_24H:F12', 'LJP007_HUVEC_24H:J13', 'LJP007_HUVEC_24H:J14', 'LJP007_HUVEC_24H:J15', 'LJP007_HUVEC_24H:J16', 'LJP007_HUVEC_24H:J17', 'LJP007_HUVEC_24H:J18', 'LJP007_HUVEC_24H:N19', 'LJP007_HUVEC_24H:N20', 'LJP007_HUVEC_24H:N21', 'LJP007_HUVEC_24H:N22', 'LJP007_HUVEC_24H:N23', 'LJP007_HUVEC_24H:N24', 'LJP008_HUVEC.A_24H:A05', 'LJP008_HUVEC.A_24H:A06', 'LJP008_HUVEC.A_24H:B05', 'LJP008_HUVEC.A_24H:B06', 'LJP008_HUVEC.A_24H:F07', 'LJP008_HUVEC.A_24H:F08', 'LJP008_HUVEC.A_24H:F09', 'LJP008_HUVEC.A_24H:F10', 'LJP008_HUVEC.A_24H:F11', 'LJP008_HUVEC.A_24H:F12', 'LJP008_HUVEC.A_24H:J13', 'LJP008_HUVEC.A_24H:J14', 'LJP008_HUVEC.A_24H:J15', 'LJP008_HUVEC.A_24H:J16', 'LJP008_HUVEC.A_24H:J17', 'LJP008_HUVEC.A

### save sig_info for specific cells

In [ ]:
idx_5 = pd.Index(level5_cell_ids, name = "level_5_sig_id")
print(idx_5)

Index(['LJP007_HUVEC_24H:A05', 'LJP007_HUVEC_24H:A06', 'LJP007_HUVEC_24H:B05',
       'LJP007_HUVEC_24H:B06', 'LJP007_HUVEC_24H:F07', 'LJP007_HUVEC_24H:F08',
       'LJP007_HUVEC_24H:F09', 'LJP007_HUVEC_24H:F10', 'LJP007_HUVEC_24H:F11',
       'LJP007_HUVEC_24H:F12',
       ...
       'REP.B024_HUVEC.A_24H:P15', 'REP.B024_HUVEC.A_24H:P16',
       'REP.B024_HUVEC.A_24H:P17', 'REP.B024_HUVEC.A_24H:P18',
       'REP.B024_HUVEC.A_24H:P19', 'REP.B024_HUVEC.A_24H:P20',
       'REP.B024_HUVEC.A_24H:P21', 'REP.B024_HUVEC.A_24H:P22',
       'REP.B024_HUVEC.A_24H:P23', 'REP.B024_HUVEC.A_24H:P24'],
      dtype='object', name='level_5_sig_id', length=8154)


In [ ]:
sig_info_cell_df = pd.DataFrame({"cell" : sig_info_df.loc[level5_cell_ids, "cell_iname"],
                                 "plate": np.zeros(len(level5_cell_ids)),
                                 "time" : sig_info_df.loc[level5_cell_ids, "pert_itime"],
                                 "level_3_samples" : sig_info_df.loc[level5_cell_ids, "distil_ids"],
                                 "samples_number" : sig_info_df.loc[level5_cell_ids, "nsample"],
                                 "pert_type" : sig_info_df.loc[level5_cell_ids, "pert_type"],
                                 "pert_drug" : sig_info_df.loc[level5_cell_ids, "cmap_name"],
                                 "targets" : np.zeros(len(level5_cell_ids)),
                                 "targets_number": np.zeros(len(level5_cell_ids)),
                                 "dose" : sig_info_df.loc[level5_cell_ids, "pert_idose"],
                                 "dose_float" : np.zeros(len(level5_cell_ids)),
                                }, index = idx_5)

# dose for DMSO = 0 uM and targets 0 ---> "DMSO_No_target"
sig_info_cell_df.loc[sig_info_cell_df[sig_info_cell_df.pert_drug == "DMSO"].index, "dose"] = "0 uM"
sig_info_cell_df.loc[sig_info_cell_df[sig_info_cell_df.pert_drug == "DMSO"].index, "targets"] = "DMSO_No_target"

for exp_id in level5_cell_ids:
    sig_info_cell_df.loc[exp_id, "plate"], a, b = exp_id.split("_")
    sig_info_cell_df.loc[exp_id, "dose_float"], c = sig_info_cell_df.loc[exp_id, "dose"].split(" ")

# dose_float dtype = object ---> dtype = int64 
sig_info_cell_df.dose_float = pd.to_numeric(sig_info_cell_df.dose_float)
# print(sig_info_cell_df.dose_float)

# targets
for exp_id in level5_cell_ids_trt_cp:
    # print(exp_id, sig_info_cell_df.loc[exp_id, "pert_drug"])
    # print(compound_info_df[compound_info_df.cmap_name == sig_info_cell_df.loc[exp_id, "pert_drug"]].target)
    # print(", ".join(compound_info_df[compound_info_df.cmap_name == sig_info_cell_df.loc[exp_id, "pert_drug"]].target))
    sig_info_cell_df.loc[exp_id, "targets"] = ", ".join(sorted(set(compound_info_df[compound_info_df.cmap_name == sig_info_cell_df.loc[exp_id, "pert_drug"]].target)))
    if sig_info_cell_df.loc[exp_id, "targets"] != "":
        sig_info_cell_df.loc[exp_id, "targets_number"] = len(set(compound_info_df[compound_info_df.cmap_name == sig_info_cell_df.loc[exp_id, "pert_drug"]].target))

display(sig_info_cell_df)

C:\Users\annas\AppData\Local\Temp\ipykernel_19920\1560421894.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'DMSO_No_target' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  sig_info_cell_df.loc[sig_info_cell_df[sig_info_cell_df.pert_drug == "DMSO"].index, "targets"] = "DMSO_No_target"
C:\Users\annas\AppData\Local\Temp\ipykernel_19920\1560421894.py:19: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'LJP007' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  sig_info_cell_df.loc[exp_id, "plate"], a, b = exp_id.split("_")
C:\Users\annas\AppData\Local\Temp\ipykernel_19920\1560421894.py:20: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0' has dtype incompatible with float64, please explicitly c

,cell,plate,time,level_3_samples,samples_number,pert_type,pert_drug,targets,targets_number,dose,dose_float
level_5_sig_id,,,,,,,,,,,
LJP007_HUVEC_24H:A05,HUVEC,LJP007,24 h,LJP007_HUVEC_24H_X1_B21:A05|LJP007_HUVEC_24H_X...,3,ctl_vehicle,DMSO,DMSO_No_target,0.0,0 uM,0.00
LJP007_HUVEC_24H:A06,HUVEC,LJP007,24 h,LJP007_HUVEC_24H_X1_B21:A06|LJP007_HUVEC_24H_X...,3,ctl_vehicle,DMSO,DMSO_No_target,0.0,0 uM,0.00
LJP007_HUVEC_24H:B05,HUVEC,LJP007,24 h,LJP007_HUVEC_24H_X1_B21:B05|LJP007_HUVEC_24H_X...,3,ctl_vehicle,DMSO,DMSO_No_target,0.0,0 uM,0.00
LJP007_HUVEC_24H:B06,HUVEC,LJP007,24 h,LJP007_HUVEC_24H_X1_B21:B06|LJP007_HUVEC_24H_X...,3,ctl_vehicle,DMSO,DMSO_No_target,0.0,0 uM,0.00
LJP007_HUVEC_24H:F07,HUVEC,LJP007,24 h,LJP007_HUVEC_24H_X1_B21:F07|LJP007_HUVEC_24H_X...,3,ctl_vehicle,DMSO,DMSO_No_target,0.0,0 uM,0.00
...,...,...,...,...,...,...,...,...,...,...,...
REP.B024_HUVEC.A_24H:P20,HUVEC,REP.B024,24 h,REP.B024_HUVEC.A_24H_X2_B25:P20,1,trt_cp,sirolimus,"FKBP1A, MTOR",2.0,0.74 uM,0.74
REP.B024_HUVEC.A_24H:P21,HUVEC,REP.B024,24 h,REP.B024_HUVEC.A_24H_X2_B25:P21,1,trt_cp,sirolimus,"FKBP1A, MTOR",2.0,0.25 uM,0.25
REP.B024_HUVEC.A_24H:P22,HUVEC,REP.B024,24 h,REP.B024_HUVEC.A_24H_X2_B25:P22,1,trt_cp,sirolimus,"FKBP1A, MTOR",2.0,0.08 uM,0.08


In [ ]:
cell_list5 = list(set(sig_info_cell_df.cell))
print("Cell values in data:", len(cell_list5), cell_list5)

plate_list = sorted(list(set(sig_info_cell_df.plate)))
print("Plate values in data:", len(plate_list), plate_list)

time_list5 = list(set(sig_info_cell_df.time))
print("Time values in data:", len(time_list5), time_list5)

pert_type_list5 = list(set(sig_info_cell_df.pert_type))
print("Perturbation_type values in data:", len(pert_type_list5), pert_type_list5)

pert_drug_list5 = sorted(list(set(sig_info_cell_df.pert_drug)))
print("Perturbation_drug values in data:", len(pert_drug_list5), pert_drug_list5)

n_targets_list = list(set(sig_info_cell_df.targets_number))
print("Number of targets for one drug in data:", len(n_targets_list), n_targets_list)

n_samples_list = list(set(sig_info_cell_df.samples_number))
print("Number of samples for one signature in data:", len(n_samples_list), n_samples_list)

dose_list5 = sorted(list(set(sig_info_cell_df.dose)))
print("Dose values in data:", len(dose_list5), dose_list5)

dose_float_list = sorted(list(set(sig_info_cell_df.dose_float)))
print("Dose values in data:", len(dose_float_list), dose_float_list)

Cell values in data: 1 ['HUVEC']
Plate values in data: 20 ['LJP007', 'LJP008', 'LJP009', 'REP.A009', 'REP.A011', 'REP.A013', 'REP.A014', 'REP.A020', 'REP.A021', 'REP.A022', 'REP.A026', 'REP.A028', 'REP.B011', 'REP.B012', 'REP.B013', 'REP.B020', 'REP.B021', 'REP.B022', 'REP.B023', 'REP.B024']
Time values in data: 1 ['24 h']
Perturbation_type values in data: 2 ['ctl_vehicle', 'trt_cp']
Perturbation_drug values in data: 819 ['3-deazaadenosine', 'A-66', 'A-769662', 'ABT-202', 'ABT-724', 'ABT-737', 'ACY-1215', 'AGI-5198', 'AGI-6780', 'AGK-2', 'AICA-ribonucleotide', 'ALW-II-38-3', 'ALW-II-49-7', 'AMD-11070', 'AMG-517', 'AP-26113', 'API-001', 'ARN-509', 'AST-1306', 'AT-13387', 'AT-7867', 'ATN-161', 'AVL-292', 'AZ-20', 'AZ-628', 'AZD-1480', 'AZD-1981', 'AZD-3514', 'AZD-4547', 'AZD-5363', 'AZD-5438', 'AZD-7545', 'AZD-7687', 'AZD-7762', 'AZD-8330', 'AZD-9291', 'AZD-9668', 'BAY-61-3606', 'BAY-87-2243', 'BAY-K8644', 'BGT-226', 'BI-2536', 'BIBX-1382', 'BIX-01294', 'BIX-02189', 'BMS-299897', 'BMS-50

In [ ]:
targets_list = list(set(sig_info_cell_df.targets))
print("Targets values in data:", len(targets_list), targets_list)

samples_list = list(set(sig_info_cell_df.level_3_samples))
print("Level_3_samples in data:", len(samples_list), samples_list)

Targets values in data: 383 ['', 'TBXAS1', 'ABL1, BCR, FGFR1, FGFR2, FGFR3, FGFR4, FLT3, KIT, RET, TEK', 'ABL1, PDGFRB', 'HDAC1, MDM2', 'IMPA1', 'ATP4A', 'EGFR, ERBB2', 'BIRC2, XIAP', 'PIK3CG', 'GSK3B', 'HTR1A', 'FKBP1A', 'DRD2, HTR2A', 'SCN1A, SCN3A, SCN5A', 'DHODH', 'CBR1', 'SLC12A1, SLC12A2', 'HTR1D, HTR2A, HTR2B, HTR2C', 'DGAT1', 'PIK3CD, PIK3CG', 'DDR1', 'DRD2, DRD3', 'ACACA, ACACB', 'F10', 'CYP1A2, CYP2B6, CYP2C19, CYP2C9, CYP2D6, CYP3A4', 'RAF1', 'HRH1, HRH3', 'ATR', 'CHRM2, SLC6A2, SLC6A4', 'ADRB2, DRD1, DRD2, HRH1, HTR1A, HTR1B, HTR2A', 'ERBB2', 'KCNJ10, SLC6A2, SLC6A4', 'DHODH, IL17A', 'HSPA1A', 'AMY2A, GAA, MGAM', 'TPO', 'HDAC1, HDAC2, HDAC3, HDAC6, HDAC8', 'ABL1, BCR, CAMK2G, HCK, LYN, SRC', 'ADRA1A', 'TGFBR1', 'CDK7', 'KCNQ2, KCNQ3, KCNQ4, KCNQ5', 'MAP2K5, MAPK7', 'SLC29A1', 'BTK', 'INS', 'AXL, MERTK, MET, MST1R, TYRO3', 'CNR2, GPR119', 'ROCK1, ROCK2', 'TXN, TXNRD1', 'ESR2', 'P2RY12', 'JAK3', 'CDK1, CDK2, CDK4, NTRK1', 'PRKCB', 'IDH1', 'PTGS1', 'FLT1, FLT3, FLT4, KDR, KIT,

In [ ]:
sig_info_cell_df.to_excel(os.path.join(out_dir, "sig_info_" + data_year + "_" + cell_name + ".xlsx"))
sig_info_cell_df.to_csv(os.path.join(out_dir, "sig_info_" + data_year + "_" + cell_name + ".csv"))

## Transform data

In [ ]:
j = []

for exp_id in level5_cell_ids:
    j += sig_info_df[sig_info_df.index == exp_id].distil_ids.item().split("|")

print(len(j), j)
print(len(set(j)), set(j))
print(len(list(set(j))), list(set(j)))

15276 ['LJP007_HUVEC_24H_X1_B21:A05', 'LJP007_HUVEC_24H_X2_B21:A05', 'LJP007_HUVEC_24H_X3.A2_B20:A05', 'LJP007_HUVEC_24H_X1_B21:A06', 'LJP007_HUVEC_24H_X2_B21:A06', 'LJP007_HUVEC_24H_X3.A2_B20:A06', 'LJP007_HUVEC_24H_X1_B21:B05', 'LJP007_HUVEC_24H_X2_B21:B05', 'LJP007_HUVEC_24H_X3.A2_B20:B05', 'LJP007_HUVEC_24H_X1_B21:B06', 'LJP007_HUVEC_24H_X2_B21:B06', 'LJP007_HUVEC_24H_X3.A2_B20:B06', 'LJP007_HUVEC_24H_X1_B21:F07', 'LJP007_HUVEC_24H_X2_B21:F07', 'LJP007_HUVEC_24H_X3.A2_B20:F07', 'LJP007_HUVEC_24H_X1_B21:F08', 'LJP007_HUVEC_24H_X2_B21:F08', 'LJP007_HUVEC_24H_X3.A2_B20:F08', 'LJP007_HUVEC_24H_X1_B21:F09', 'LJP007_HUVEC_24H_X2_B21:F09', 'LJP007_HUVEC_24H_X3.A2_B20:F09', 'LJP007_HUVEC_24H_X1_B21:F10', 'LJP007_HUVEC_24H_X2_B21:F10', 'LJP007_HUVEC_24H_X3.A2_B20:F10', 'LJP007_HUVEC_24H_X1_B21:F11', 'LJP007_HUVEC_24H_X2_B21:F11', 'LJP007_HUVEC_24H_X3.A2_B20:F11', 'LJP007_HUVEC_24H_X1_B21:F12', 'LJP007_HUVEC_24H_X2_B21:F12', 'LJP007_HUVEC_24H_X3.A2_B20:F12', 'LJP007_HUVEC_24H_X1_B21:J13', 'L

In [ ]:
Data_mean = pd.DataFrame(index = level5_cell_ids, columns = landmark_gene_symbols)

for exp_id in level5_cell_ids:
    Data_mean.loc[exp_id,:] = level3_cell_lm_df.loc[
                                                    sig_info_df[sig_info_df.index == exp_id].distil_ids
                                                    .item().split("|")
                                                    ].mean(axis = "rows")

display(Data_mean)

,AARS,ABCB6,ABCC5,ABCF1,ABCF3,ABHD4,ABHD6,ABL1,ACAA1,ACAT2,...,ZMIZ1,ZMYM2,ZNF131,ZNF274,ZNF318,ZNF395,ZNF451,ZNF586,ZNF589,ZW10
LJP007_HUVEC_24H:A05,8.6543,7.2702,5.421567,10.206284,8.668567,6.291083,7.29735,10.2665,7.886866,7.369333,...,9.997766,8.924283,8.172851,10.410832,6.417284,8.868833,8.270333,7.877966,6.597134,9.478184
LJP007_HUVEC_24H:A06,8.83105,7.528433,5.40225,10.1914,8.61905,6.124667,6.283233,10.252467,7.945833,8.217366,...,9.891067,8.6332,8.052533,10.540349,6.791883,8.884767,8.01475,7.95005,6.4894,10.37965
LJP007_HUVEC_24H:B05,8.6362,7.112067,6.522367,10.412601,8.625466,7.8146,6.352533,10.222934,7.659467,7.17925,...,9.816334,9.1984,8.3492,10.625967,6.912067,9.525933,8.259167,5.931117,6.772984,9.358001
LJP007_HUVEC_24H:B06,8.562317,7.0633,5.246216,10.193,8.260667,5.878483,6.140684,10.194533,7.775967,8.232682,...,10.5703,8.936366,8.444584,9.891833,6.655167,9.194217,8.20435,6.088717,6.37025,9.682099
LJP007_HUVEC_24H:F07,8.848016,7.3998,5.387566,10.242167,8.757466,6.069233,7.42895,10.321234,7.831434,6.688283,...,10.353883,8.932067,8.1465,10.353667,6.580184,8.591983,8.357217,5.825783,6.678367,10.8229
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
REP.B024_HUVEC.A_24H:P20,7.9361,7.787575,4.473175,11.2182,8.547501,7.270425,6.112675,10.417875,6.01755,7.00955,...,11.05645,6.5784,8.3508,10.048349,6.05175,10.78325,6.31035,5.431025,5.431025,8.873125
REP.B024_HUVEC.A_24H:P21,8.94175,6.48815,4.760575,11.20965,8.474951,9.084325,10.2366,10.04525,7.93295,7.247825,...,10.3146,6.0433,7.734674,10.11285,5.910475,10.7208,6.69585,11.254049,6.0433,8.55265
REP.B024_HUVEC.A_24H:P22,9.23155,6.5445,4.4451,9.56935,8.28555,7.111125,6.703775,10.0708,6.924125,7.61,...,9.7997,5.72245,7.871575,10.38735,6.06545,11.05345,6.139925,5.439625,5.93565,9.1034
REP.B024_HUVEC.A_24H:P23,9.348249,7.52125,4.7031,10.966,8.255275,6.92015,5.936725,9.994275,6.74905,7.350125,...,10.0047,6.5078,7.987,10.073975,6.799675,10.8217,6.76605,5.29935,5.8325,8.5198


In [ ]:
Data_mean.to_excel(os.path.join(out_dir, "Data_mean_" + data_year + "_" + cell_name + ".xlsx"))
Data_mean.to_csv(os.path.join(out_dir, "Data_mean_" + data_year + "_" + cell_name + ".csv"))

## time = 24 h only

In [ ]:
Data_controls = Data_mean.loc[level5_cell_ids_ctl]
display(Data_controls)

Plate_MeanControl_df = pd.DataFrame(index = plate_list, columns = landmark_gene_symbols)

for plate_id in plate_list:  
    Plate_MeanControl_df.loc[plate_id,:] = Data_controls.filter(regex = plate_id + "_" + cell_name + ".*", axis = "rows").mean(axis = "rows")

display(Plate_MeanControl_df)

,AARS,ABCB6,ABCC5,ABCF1,ABCF3,ABHD4,ABHD6,ABL1,ACAA1,ACAT2,...,ZMIZ1,ZMYM2,ZNF131,ZNF274,ZNF318,ZNF395,ZNF451,ZNF586,ZNF589,ZW10
LJP007_HUVEC_24H:A05,8.6543,7.2702,5.421567,10.206284,8.668567,6.291083,7.29735,10.2665,7.886866,7.369333,...,9.997766,8.924283,8.172851,10.410832,6.417284,8.868833,8.270333,7.877966,6.597134,9.478184
LJP007_HUVEC_24H:A06,8.83105,7.528433,5.40225,10.1914,8.61905,6.124667,6.283233,10.252467,7.945833,8.217366,...,9.891067,8.6332,8.052533,10.540349,6.791883,8.884767,8.01475,7.95005,6.4894,10.37965
LJP007_HUVEC_24H:B05,8.6362,7.112067,6.522367,10.412601,8.625466,7.8146,6.352533,10.222934,7.659467,7.17925,...,9.816334,9.1984,8.3492,10.625967,6.912067,9.525933,8.259167,5.931117,6.772984,9.358001
LJP007_HUVEC_24H:B06,8.562317,7.0633,5.246216,10.193,8.260667,5.878483,6.140684,10.194533,7.775967,8.232682,...,10.5703,8.936366,8.444584,9.891833,6.655167,9.194217,8.20435,6.088717,6.37025,9.682099
LJP007_HUVEC_24H:F07,8.848016,7.3998,5.387566,10.242167,8.757466,6.069233,7.42895,10.321234,7.831434,6.688283,...,10.353883,8.932067,8.1465,10.353667,6.580184,8.591983,8.357217,5.825783,6.678367,10.8229
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
REP.B024_HUVEC.A_24H:J14,9.636475,8.1979,5.04565,10.80415,9.1167,7.0574,9.969999,10.054874,7.70995,3.8689,...,10.5372,8.04495,7.7907,11.1474,5.9908,8.679399,6.91795,5.9333,7.5334,9.83695
REP.B024_HUVEC.A_24H:J15,9.749849,7.72455,5.0341,10.32305,8.7761,7.012675,10.36025,10.05835,6.988025,8.40475,...,11.150775,7.896175,7.27685,10.94935,5.7232,8.26885,6.4546,11.711849,6.5115,9.54385
REP.B024_HUVEC.A_24H:J16,10.2222,7.329299,4.601675,11.7438,8.59135,7.19605,10.29595,8.423625,6.64665,3.7162,...,10.168949,7.0496,7.76135,11.2809,6.11355,8.24395,6.70865,12.280001,6.586525,8.962151
REP.B024_HUVEC.A_24H:J17,10.067474,7.25705,4.929225,10.2278,8.80895,6.9134,10.104549,8.727625,7.89355,8.7578,...,10.32185,6.8401,7.7592,10.7596,6.0509,8.5277,7.23865,5.36315,6.72225,9.357201


,AARS,ABCB6,ABCC5,ABCF1,ABCF3,ABHD4,ABHD6,ABL1,ACAA1,ACAT2,...,ZMIZ1,ZMYM2,ZNF131,ZNF274,ZNF318,ZNF395,ZNF451,ZNF586,ZNF589,ZW10
LJP007,8.772834,7.376733,5.974015,10.342377,8.68237,6.364067,6.387905,10.201456,7.907519,8.030816,...,10.162137,8.777746,8.136734,10.295532,6.560993,9.185872,8.163544,6.554166,6.680099,9.869844
LJP008,9.162585,7.921719,5.646467,10.387697,8.460045,6.429663,6.389883,10.16632,7.042249,8.980617,...,10.032756,9.056445,8.119501,10.520768,6.421531,10.35859,8.416316,6.401732,7.137694,9.90628
LJP009,9.545461,8.17295,5.54637,10.814081,8.567252,6.478401,6.338872,10.669367,7.349819,8.739348,...,10.411063,9.281416,8.393112,11.265049,6.590534,9.51554,8.569538,6.268349,7.063978,10.018079
REP.A009,9.466514,7.670481,5.144688,10.53926,8.700202,6.843473,7.09727,9.976118,6.951096,9.480893,...,10.645129,9.050173,8.156075,11.510683,6.324236,8.875551,8.314677,7.02701,7.962832,9.855529
REP.A011,9.557984,8.78072,8.056558,10.32072,7.510651,6.739797,8.332405,9.277649,6.566989,6.85489,...,10.104499,8.054876,8.333851,10.460821,6.422643,9.5454,7.885105,10.265236,7.96451,10.445672
REP.A013,9.65677,9.750756,7.868852,9.801272,7.315505,6.984773,9.134728,9.607476,5.758965,6.622519,...,8.993827,7.444775,8.19305,10.468292,7.30653,9.871197,8.032331,9.805044,6.587114,10.024111
REP.A014,9.992128,9.499233,8.954712,9.45041,7.65704,7.12405,8.023488,9.995618,6.80765,6.568355,...,10.247635,8.138973,8.630657,8.312538,6.971827,10.089655,8.90447,9.650833,7.330228,9.711457
REP.A020,9.370823,8.084358,5.481066,10.604104,8.270334,7.105903,7.300906,9.475017,6.780558,6.384887,...,10.484247,6.947559,8.328162,10.856918,6.059272,9.900292,7.015903,6.992774,6.509831,9.023388
REP.A021,9.855112,8.108882,5.163878,10.633627,8.53849,7.217458,7.023047,9.582188,7.101396,7.205605,...,10.615843,7.015331,8.476066,10.247782,6.170798,9.891266,7.115764,6.641107,6.85553,9.005927
REP.A022,9.159049,8.19277,5.610581,10.833672,8.874631,6.782389,6.97874,9.549154,6.507109,6.560314,...,10.604491,6.965476,7.807751,12.223845,6.04418,10.992577,6.587641,6.369236,6.552436,9.27463


In [ ]:
Data_norm = pd.DataFrame(index = level5_cell_ids, columns = landmark_gene_symbols)

for exp_id in level5_cell_ids:
    plate_id, cell, tale = exp_id.split("_")
    Data_norm.loc[exp_id, :] = pd.to_numeric(Data_mean.loc[exp_id, :] - Plate_MeanControl_df.loc[plate_id, :])

display(Data_norm)

,AARS,ABCB6,ABCC5,ABCF1,ABCF3,ABHD4,ABHD6,ABL1,ACAA1,ACAT2,...,ZMIZ1,ZMYM2,ZNF131,ZNF274,ZNF318,ZNF395,ZNF451,ZNF586,ZNF589,ZW10
LJP007_HUVEC_24H:A05,-0.118534,-0.106533,-0.552448,-0.136094,-0.013804,-0.072984,0.909445,0.065044,-0.020653,-0.661483,...,-0.164372,0.146537,0.036117,0.1153,-0.143709,-0.317039,0.106789,1.323801,-0.082965,-0.39166
LJP007_HUVEC_24H:A06,0.058216,0.1517,-0.571765,-0.150978,-0.06332,-0.239401,-0.104672,0.051012,0.038314,0.18655,...,-0.271071,-0.144547,-0.084201,0.244817,0.230891,-0.301105,-0.148794,1.395884,-0.190699,0.509806
LJP007_HUVEC_24H:B05,-0.136634,-0.264667,0.548352,0.070223,-0.056904,1.450533,-0.035372,0.021478,-0.248052,-0.851566,...,-0.345803,0.420653,0.212466,0.330435,0.351074,0.340061,0.095622,-0.623049,0.092885,-0.511843
LJP007_HUVEC_24H:B06,-0.210517,-0.313434,-0.727799,-0.149377,-0.421704,-0.485584,-0.247222,-0.006922,-0.131553,0.201866,...,0.408163,0.15862,0.30785,-0.403699,0.094174,0.008345,0.040805,-0.465449,-0.309849,-0.187745
LJP007_HUVEC_24H:F07,0.075182,0.023066,-0.586449,-0.100211,0.075096,-0.294834,1.041045,0.119778,-0.076086,-1.342532,...,0.191746,0.154321,0.009766,0.058135,0.019191,-0.593889,0.193672,-0.728383,-0.001732,0.953056
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
REP.B024_HUVEC.A_24H:P20,-1.905539,-0.0073,-0.731396,0.430556,-0.123994,-0.124384,-2.507582,0.673428,-0.800314,1.072226,...,0.379562,-0.212133,0.529867,-0.525423,-0.088439,0.87156,-0.600894,-3.59726,-1.67963,-0.28819
REP.B024_HUVEC.A_24H:P21,-0.89989,-1.306725,-0.443996,0.422006,-0.196544,1.689516,1.616344,0.300803,1.115086,1.310501,...,-0.362287,-0.747232,-0.086258,-0.460922,-0.229714,0.80911,-0.215394,2.225764,-1.067355,-0.608665
REP.B024_HUVEC.A_24H:P22,-0.61009,-1.250375,-0.759471,-1.218294,-0.385945,-0.283684,-1.916481,0.326352,0.106261,1.672676,...,-0.877188,-1.068082,0.050643,-0.186422,-0.074739,1.14176,-0.771319,-3.58866,-1.175005,-0.057915
REP.B024_HUVEC.A_24H:P23,-0.49339,-0.273625,-0.501471,0.178356,-0.41622,-0.474659,-2.683532,0.249828,-0.068814,1.412802,...,-0.672188,-0.282732,0.166068,-0.499798,0.659486,0.91001,-0.145194,-3.728935,-1.278155,-0.641515


In [ ]:
Data_norm.to_excel(os.path.join(out_dir, "Data_norm_" + data_year + "_" + cell_name + ".xlsx"))
Data_norm.to_csv(os.path.join(out_dir, "Data_norm_" + data_year + "_" + cell_name + ".csv"))